In [3]:
# Загружаем данные
import pandas as pd

from utils import load_parquet_from_s3

items = pd.read_parquet(load_parquet_from_s3("recsys/data/items.parquet"))
print(items.shape)

events = pd.read_parquet(load_parquet_from_s3("recsys/data/events.parquet"))
print(events.shape)


(980970, 4)
(188885672, 5)


In [ ]:
# Разделим данные
import sklearn.preprocessing

train_test_global_time_split_date = pd.to_datetime("2022-12-16")
train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]
print(events_train.shape, events_test.shape)

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])
print(events_train['item_id_enc'].max())


(178811394, 5) (10074278, 5)


/tmp/ipykernel_100527/3072731248.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/tmp/ipykernel_100527/3072731248.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])
/tmp/ipykernel_100527/3072731248.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

980969


/tmp/ipykernel_100527/3072731248.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])


In [6]:
# Вычислим размер матрицы
x = events_train['item_id_enc'].nunique()
print(x)
y = events_train['user_id_enc'].nunique()
print(y)
r = ((x * y) / 1024 ** 3)
print(r)

979546
475751
434.0149360205978


In [9]:
# Создадим матрицу
import numpy as np
import scipy

user_item_matrix_train = scipy.sparse.csr_matrix((
    np.ones(len(events_train), dtype=np.int8),  # Значения: 1 для каждой пары
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)
print('Матрица создана!')

Матрица создана!


In [10]:
# Создадим и натренируем модель
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

/home/mle-user/mle_projects/mle-recsys-project-start/env_recsys_start/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mle-user/mle_projects/mle-recsys-project-start/env_recsys_start/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 50/50 [37:29<00:00, 45.00s/it]


In [11]:
# Получим набор похожих объектов в similar_items

# получим энкодированные идентификаторы всех объектов, известных нам из events_train
train_item_ids_enc = events_train['item_id_enc'].unique()

max_similar_items = 10

# получаем списки похожих объектов, используя ранее полученную ALS-модель
# метод similar_items возвращает и сам объект, как наиболее похожий
# этот объект мы позже отфильтруем, но сейчас запросим на 1 больше
similar_items = als_model.similar_items(train_item_ids_enc, N=max_similar_items+1)

# преобразуем полученные списки в табличный формат
sim_item_item_ids_enc = similar_items[0]
sim_item_scores = similar_items[1]

similar_items = pd.DataFrame({
    "item_id_enc": train_item_ids_enc,
    "sim_item_id_enc": sim_item_item_ids_enc.tolist(), 
    "score": sim_item_scores.tolist()})
similar_items = similar_items.explode(["sim_item_id_enc", "score"], ignore_index=True)

# приводим типы данных
similar_items["sim_item_id_enc"] = similar_items["sim_item_id_enc"].astype("int")
similar_items["score"] = similar_items["score"].astype("float")

# получаем изначальные идентификаторы
similar_items["item_id_1"] = item_encoder.inverse_transform(similar_items["item_id_enc"])
similar_items["item_id_2"] = item_encoder.inverse_transform(similar_items["sim_item_id_enc"])
similar_items = similar_items.drop(columns=["item_id_enc", "sim_item_id_enc"])

# убираем пары с одинаковыми объектами
similar_items = similar_items.query("item_id_1 != item_id_2")
print('Готово!')

Готово!


In [12]:
# Сохраним набор похожих объектов
import os
import io
import boto3
from dotenv import load_dotenv

load_dotenv()

S3_ENDPOINT_URL = "https://storage.yandexcloud.net"
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
S3_BUCKET_NAME = os.getenv("S3_BUCKET_NAME")

# Путь к файлу на S3
s3_key = "recsys/recommendations/similar_items.parquet"

s3_client = boto3.client("s3",
    endpoint_url=S3_ENDPOINT_URL,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)

# Сохраняем
buffer = io.BytesIO()
similar_items.to_parquet(buffer, index=False)
buffer.seek(0)

# Загружаем файл на S3
s3_client.put_object(
    Bucket=S3_BUCKET_NAME,
    Key=s3_key,
    Body=buffer
)

print(f"Файл успешно сохранен на S3 по пути: {s3_key}")

Файл успешно сохранен на S3 по пути: recsys/recommendations/similar_items.parquet


In [2]:
# Загружаем данные
import pandas as pd

from utils import load_parquet_from_s3

similar_items = pd.read_parquet(load_parquet_from_s3("recsys/recommendations/similar_items.parquet"))
print(similar_items.shape)
print(similar_items.head(2))



(9796212, 3)
      score  item_id_1  item_id_2
0  0.978334        966   37408173
1  0.974815        966   37408181


In [14]:
events['user_id'].unique()
print(events['user_id'].unique()[:10])

[ 4 13 14 16 19 22 23 24 25 26]


In [4]:
# Укажите идентификатор объекта, наиболее похожего на объект
filtered_items = similar_items[similar_items['item_id_1'] == 53404]
print(filtered_items)

         score  item_id_1  item_id_2
3770  0.882784      53404      96089
3771  0.877298      53404      37384
3772  0.873760      53404     178529
3773  0.873183      53404    6705392
3774  0.867275      53404     148345
3775  0.831046      53404      48951
3776  0.823033      53404      53412
3777  0.811312      53404      36246
3778  0.778754      53404     178495
3779  0.777983      53404     694683
